# GTFS Onboarding Exercise

The [MBTA’s GTFS](https://github.com/mbta/gtfs-documentation/blob/master/reference/gtfs.md) Feed consists of a series of [text files in a standard format](https://developers.google.com/transit/gtfs/reference) that represent the system’s static schedule (along with other geographic information). Throughout this guide, you will gain familiarity with the data tables included in the feed. Additionally, you can find links to the corresponding resource in the MBTA's V3 API. For the purposes of this guide, we will be using the GTFS Feed active on [October 14, 2020](https://cdn.mbta.com/archive/archived_feeds.txt).

At the end of each section, there will be a question to test your understanding. If you'd like to see an example solution, click the ellipsis under "Reveal Solution"

![](img/reveal.png)

## Resources
* [GTFS Documentation](https://github.com/google/transit/blob/master/gtfs/spec/en/reference.md)
* [MBTA GTFS Documentation](https://github.com/mbta/gtfs-documentation/blob/master/reference/gtfs.md)
* [MBTA V3 API Documentation](https://api-v3.mbta.com/docs/swagger/index.html)

## Getting Oriented

### Jupypter Notebook
You can simply follow along with this guide and run the sample code by selecting code cells and pressing the "Run" button in the toolbar or pressing `Shift + Return`. To learn more about the Jupyter Notebook and its ecoystem, see its [documentation](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html).

### SQL
An SQLite database with tables corresponding to each table in the [MBTA GTFS Documenation](https://github.com/mbta/gtfs-documentation/blob/master/reference/gtfs.md) has been loaded for your convenience. Data exploration will be performed in SQL throughout this guide.

In [ ]:
## Select this cell and press Shift + Return

%load_ext sql
%sql sqlite:///feed.db

You can now execute SQL queries by adding `%%sql` to the top of code cells.

In [ ]:
%%sql
SELECT *
FROM feed_info;

### Python
Additionally, you can execute arbitrary Python code if you would rather explore the files that way. The `pandas` library has been imported for your convenience. 

In [ ]:
## Select this cell and press Shift + Return
import pandas as pd
print(2 + 2)

### Raw Data
The raw `.txt` file are available to download or inspect in the `feed` directory. For example, see [/feed/feed_info.txt](feed/feed_info.txt). Feel free to explore these files in Excel, Google Sheets, or your favorite spreadsheet software.

## Services
*A set of dates on which trips run for one or more routes*

**GTFS Tables**: [`calendar`](feed/calendar.txt), [`calendar_dates`](feed/calendar_dates.txt), [`calendar_attributes`](feed/calendar_attributes.txt)

**MBTA V3 API Resource**: [`Service`](https://api-v3.mbta.com/docs/swagger/index.html#/Service/ApiWeb_ServiceController_index)

When a rider is trying to figure out if and when a particular route is running trips, they probably have to ask themselves a few questions:
* What is the day of week?
* What's the date?
* Is it a holiday?

As an example, let's look at the 89 bus using MBTA.com's [Schedule Finder](https://www.mbta.com/schedules/89/line?schedule_direction%5Bdirection_id%5D=0&schedule_direction%5Borigin%5D=place-sull), or the [PDF schedule](https://cdn.mbta.com/sites/default/files/route_pdfs/2020-fall/R089.pdf). On MBTA.com, note the four different schedules to choose from (pictured below):
* Weekday Schedule (Fall)
* Saturday Schedule (Fall)
* Sunday Schedule (Fall)
* Thanksgiving Day, Nov 26

![](img/services.png)

As you can see in the Schedule Finder and PDF schedules, the major difference between these services is in the number and frequency of scheduled trips, though in some cases the pattern, or combination of stops, can also vary. If you've ever heard someone refer to the "Fall weekday schedule" or a "Holiday schedule," they referring to this concept of a service.

So how do we know on which dates these services are active and running trips, and how is this represented in our GTFS data? Through a combination of the data in the `calendar`, `calendar_attributes`, and `calendar_dates` tables.

### calendar
First, let's take a look at the first few rows of the `calendar` table.

In [ ]:
%%sql
SELECT *
FROM calendar
LIMIT 3;

The `service_id` is just a unique identifier for a service.

For each column representing a day of week (`monday`, `tuesday`, etc.), a value of `1` indicates that the service **does** run trips on that day of week, and a value of `0` indicates that the service **does not** run trips on that day of week.

The `start_date` and `end_date` fields indicate the date range for which the service is active. 

So looking at service `BUS420-10-Wdy-02`, we can say it runs trips on weekdays only from October 2, 2020 to December 18, 2020.

So if we wanted to services running trips only on weekdays, we could run this query:

In [ ]:
%%sql
SELECT *
FROM calendar
WHERE monday = 1
AND tuesday = 1
AND wednesday = 1
AND thursday = 1
AND friday = 1
AND saturday = 0
AND sunday = 0
ORDER BY service_id
LIMIT 3;

And if we want to find services ending before December 1, 2020, we could run this query:

In [ ]:
%%sql
SELECT *
FROM calendar
WHERE end_date < "20201201"
ORDER BY service_id
LIMIT 3;

### calendar_attributes
We include additional metadata about services in the `calendar_attributes` table. Let's take a look at the first few rows.

In [ ]:
%%sql
SELECT *
FROM calendar_attributes
LIMIT 3;

As you can see, this table includes a `service_id` column, which corresponds to a row in the `calendar` table, so we can combine these two tables to view human readable names and descriptions for each service.

In [ ]:
%%sql
SELECT *
FROM calendar
JOIN calendar_attributes on calendar.service_id = calendar_attributes.service_id
LIMIT 3;


### calendar_dates
Rows in the `calendar_dates` table define exceptions to the default service patterns defined in the `calendar` table. Let's take a look at the first few rows.

In [ ]:
%%sql
SELECT *
FROM calendar_dates
LIMIT 10;

Like `calendar_attributes`, the `service_id` in `calendar_dates` corresponds to a row in the `calendar` table. 

An `exception_type` of `1` indicates that we have added the service and its trips on the specified `date`.

An `exception_type` of `2` indicates that we have removed the service and its trips on the specified `date`.

As seen above, on October 16, 2020, we added service `RTL420-5-Wdy-01-OrgSkpNrhStnNrd` and removed service `RTL420-5-Wdy-01`.

So now that we can identify which services are running trips on a given date, how can we identify *which* trips for *which* routes are running?

### Test Your Knowledge
What is the `service_id`, `servicde_description`, and `end_date` for the Saturday service has the earliest end date?

In [ ]:
%%sql

/* Your code here */

Reveal solution

In [ ]:
%%sql

SELECT c.service_id, ca.service_description, c.end_date
FROM calendar as c
JOIN calendar_attributes as ca
ON c.service_id = ca.service_id
WHERE monday = 0
AND tuesday = 0
AND wednesday = 0
AND thursday = 0
AND friday = 0
AND saturday = 1
AND sunday = 0
ORDER BY end_date
LIMIT 1;



Which service has the most days added to the feed as exceptions?

In [ ]:
%%sql

/* Your code here */

Reveal solution

In [ ]:
%%sql
SELECT service_id, COUNT(*) as added_days
FROM calendar_dates
WHERE exception_type = 1
GROUP BY service_id
ORDER BY added_days DESC
LIMIT 1;

## Routes
*Metadata about individual routes*

**GTFS Table:** [routes](feed/routes.txt)

**MBTA V3 API Resource**: [Route](https://api-v3.mbta.com/docs/swagger/index.html#/Route/ApiWeb_RouteController_show)

First, let's take a look at the first few rows of the `routes` table.

In [ ]:
%%sql
SELECT *
FROM routes
LIMIT 5;

`route_id` is a unique identifier for individual routes, which is referenced in other tables, much like `service_id` is referenced in `calendar_attributes` and `calendar_dates`.

`route_desc` categorizes a route's level of service. Possible values for the MBTA implementation include:
* `Local Bus`
* `Inner Express`
* `Outer Express`
* `Rapid Transit`
* `Commuter Rail`
* `Ferry`
* `Free`
* `Special`


Let's look at the entry for the 89 bus.

In [ ]:
%%sql
SELECT *
FROM routes
WHERE route_id = "89";

Each individual route has one entry in the `routes` table. But you may be wondering, how do we represent the different patterns a route may take, both varying by direction (e.g. Inbound vs Outbound) and in the combination of stops (like the Red Line or 89 bus), and how do we know which pattern a given trip follows? The simplest way is to identify a given trip's `route_pattern`.

### Test Your Knowledge
How many routes are categorized as `Local Bus`?

In [ ]:
%%sql 

/* Your code here */

Reveal solution

In [ ]:
%%sql
SELECT COUNT(*)
FROM routes
WHERE route_desc = "Local Bus";

## Route Patterns
*Subsets of a route, representing different possible patterns of where trips may serve*

**GTFS Table**: [route_patterns](feed/route_patterns.txt)

**MBTA V3 API Resource**: [RoutePattern](https://api-v3.mbta.com/docs/swagger/index.html#/RoutePattern/ApiWeb_RoutePatternController_show)

Let's take a look at the first few rows of the `route_patterns` table.

In [ ]:
%%sql

SELECT * 
FROM route_patterns
LIMIT 10;

As you can see, there are multiple route patterns per route. 

For the `Orange` route, there are two patterns:
* `Orange-3-0` for the Oak Grove to Forest Hills pattern
* `Orange-5-0` for the Forest Hills to Oak Grove pattern

For the `Red` route, there are route patterns for each direction and combination of stops:
* `Red-1-0` for Alewife to Ashmont
* `Red-1-1` for Ashmont to Alewife
* `Red-3-0` for Alewife to Braintree
* `Red-3-1` for Braintree to Alewife
* `Red-3-0_70101_70105_0` for Alewife to Quincy Center (as part of shuttling)
* `Red-3-1_70105_70102_2` for Quincy Center to Alewife (as part of shuttling)

Similarly, you can see four patterns for the 89 bus, which corresponds to what we see on the Schedule Viewer.

In [ ]:
%%sql

SELECT * 
FROM route_patterns
WHERE route_id = "89";

![](img/89.png)

With the combination of `service_id`, `route_id` and `route_pattern_id`, we can tell which routes are running which trips on a given date.

### Test Your Knowledge

Which routes have exactly two patterns?

In [ ]:
%%sql

/* Your code here */

Reveal solution

In [ ]:
%%sql
SELECT route_id
FROM (
    SELECT route_id, COUNT(*) as n_patterns
    FROM route_patterns
    GROUP BY route_id
)
WHERE n_patterns = 2

## Trips
*The journey of a particular vehicle through a set of stops*

**GTFS Table**: [trips](feed/trips.txt)

**MBTA V3 API Resource**: [`Trip`](https://api-v3.mbta.com/docs/swagger/index.html#/Trip/ApiWeb_TripController_show)

The `trips` table contains metadata for each individual trip in a GTFS feed. Let's look at the first few rows.

In [ ]:
%%sql
SELECT *
FROM trips
LIMIT 3;


Once again, we see a `service_id` column corresponding to a row in the `calendar` table. 

Additionally, we see:
* `trip_id`: a unique identifier for the trip
* `route_id`: a unique identifier for a route in the `routes` table, which contains metadata about each route
* `route_pattern_id`: a unique identifier for a pattern in the `route_patterns` table, which contains metadata about each combination of stops a route may take

Let's take a look at all of the services which include trips by the 89 bus.

In [ ]:
%%sql
SELECT DISTINCT service_id
FROM trips
WHERE route_id = "89";

As a reminder, these services can include trips on multiple routes.

In [ ]:
%%sql
SELECT COUNT(*)
FROM (
    SELECT DISTINCT route_id
    FROM trips
    WHERE service_id == "FallWeekday"
);

Now, let's take a look at all of the trips for the 89 bus on the `FallWeekday` service.

In [ ]:
%%sql
SELECT *
FROM trips
WHERE service_id == "FallWeekday" AND route_id = "89";

Finally, let's look at only trips on the Sullivan to Davis Square pattern.

In [ ]:
%%sql
SELECT *
FROM trips
WHERE service_id == "FallWeekday" AND route_pattern_id = "89-2-0";

### Test Your Knowledge
For each service running 89 trips, how many trips are on the Sullivan to Clarendon Hill pattern?

In [ ]:
%%sql
/* Your code here */

Reveal solution

In [ ]:
%%sql
SELECT service_id, COUNT(*) as n_trips
FROM TRIPS
WHERE route_pattern_id = "89-_-0"
GROUP BY service_id;

## Schedules
*When a vehicle should arrive and depart a given stop for a given trip*

**GTFS Table**: [`stop_times`](feed/stop_times.txt)

**MBTA V3 API Resource**: [`Schedule`](https://api-v3.mbta.com/docs/swagger/index.html#/Schedule/ApiWeb_ScheduleController_index)

The `stop_times` table includes a row for every scheduled stop on every scheduled trip in a GTFS Feed. Let's take a look at the first few rows.

In [ ]:
%%sql
SELECT *
FROM stop_times
LIMIT 10;

Each `trip_id` corresponds to a row in the `trips` table, and there should be one row for every stop that trip makes.

The `arrival_time` and `departure_time` indicate when a vehicle should arrive and depart the stop identified by `stop_id` (more on stops later).

The `stop_sequence` field indicates the order of stops for a particular trip. The values must increase along the trip but do not need to be consecutive.

So if we have a given `trip_id`, we can see all of the stops in order for that trip.

In [ ]:
%%sql
SELECT *
FROM stop_times
WHERE trip_id = "45295219"
ORDER BY CAST(stop_sequence as int);

Metadata about the physical stops referenced by `stop_id` can be found in the `stops_table`.

### Test Your Knowledge
At what time should the trip `45692465` arrive at its final stop?

In [ ]:
%%sql

/* Your code here */

Reveal solution

In [ ]:
%%sql
SELECT arrival_time
FROM stop_times
WHERE trip_id = "45692465"
AND stop_sequence = (
    SELECT MAX(CAST(stop_sequence as int))
    FROM stop_times
    WHERE trip_id = "45692465"
    GROUP BY trip_id
)
LIMIT 10;

## Stops
*Details for a specific stop*

**GTFS Table**: [`stops`](feed/stops.txt)

**MBTA V3 API Resource**: [`Stop`](https://api-v3.mbta.com/docs/swagger/index.html#/Stop/ApiWeb_StopController_show)

Let's take a look at the first few rows of the `stops` table.

In [ ]:
%%sql
SELECT *
FROM stops
LIMIT 10;

This information can be joined with the `stop_times` table to construct a readable schedule for a given trip.

In [ ]:
%%sql
SELECT st.trip_id as trip_id, 
    st.arrival_time as arrival_time, 
    st.departure_time as departure_time, 
    st.stop_id as stop_id,
    st.stop_sequence as stop_sequence,
    s.stop_name as stop_name
FROM stop_times as st
JOIN stops as s 
ON st.stop_id = s.stop_id
WHERE trip_id = "45771950"
ORDER BY CAST(stop_sequence as int);

### Test Your Knowledge
Display the following fields for the last trip of the day for the 89 bus on the Sullivan to Clarendon Hill pattern on Fall weekdays:
* `stop_id`
* `stop_name`
* `arrival_time`
* `departure_time`

In [ ]:
%%sql

/* Your code here */

Reveal solution

In [ ]:
%%sql

SELECT st.stop_id as stop_id, s.stop_name as stop_name, MAX(st.arrival_time) as arrival_time, MAX(st.departure_time) as departure_time
FROM stop_times as st
JOIN stops as s
ON st.stop_id == s.stop_id
JOIN trips as t
ON st.trip_id == t.trip_id
WHERE t.route_id == "89"
AND t.service_id == "FallWeekday"
AND t.route_pattern_id == "89-_-1"
GROUP BY st.stop_sequence, s.stop_name
ORDER BY CAST(st.stop_sequence as int)

## What's next?
Open up a [new notebook](GTFS_Exploration.ipynb) and explore the current feed!